In [ ]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent.parent.absolute()))

In [ ]:
from multiversx_sdk_core import Address
from multiversx_sdk_core.transaction_builders import ContractCallBuilder, ESDTNFTTransferBuilder
from multiversx_sdk_core.transaction_builders import DefaultTransactionBuildersConfiguration
from multiversx_sdk_core import TokenPayment
from multiversx_sdk_network_providers import ProxyNetworkProvider
from multiversx_sdk_core import AccountNonceHolder
from multiversx_sdk_wallet import UserSigner
from pathlib import Path
from tools.notebooks import env

config = DefaultTransactionBuildersConfiguration(env.CHAIN_ID)
proxy = ProxyNetworkProvider(env.PROXY_URL)

user_signer = UserSigner.from_pem_file(Path(env.USER1_PEM))

Wrap tokens

In [ ]:
user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfer = TokenPayment.egld_from_amount(600000)

builder = ContractCallBuilder(
    config,
    contract=env.EGLD_WRAP_SH0_CONTRACT,
    function_name="wrapEgld",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    value=transfer
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
pair_contract = Address.from_bech32("erd1qqqqqqqqqqqqqpgqa0fsfshnff4n76jhcye6k7uvd7qacsq42jpsp6shh2")

transfers = [
    TokenPayment.fungible_from_amount(env.WEGLD, 600000, 18)
]

builder = ContractCallBuilder(
    config,
    contract=pair_contract,
    function_name="swapTokensFixedInput",
    caller=user,
    call_arguments=[env.MEX, 1],
    esdt_transfers=transfers,
    gas_limit=50000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Unwrap tokens

In [ ]:
user = env.USER1
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

transfers = [
    TokenPayment.fungible_from_amount(env.WEGLD, 1, 18)
]

builder = ContractCallBuilder(
    config,
    contract=env.EGLD_WRAP_SH0_CONTRACT,
    function_name="unwrapEgld",
    caller=user,
    call_arguments=[],
    gas_limit=30000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)

Token transfer

In [ ]:
transfer = TokenPayment.fungible_from_amount(env.WEGLD, 2000000, 18)

user = env.USER1
receiver = env.USER2
user_nonce_holder = AccountNonceHolder(proxy.get_account(user).nonce)

builder = ESDTNFTTransferBuilder(
    config,
    destination=receiver,
    sender=user,
    gas_limit=30000000,
    payment=transfer
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

In [ ]:
tx.nonce = user_nonce_holder.get_nonce_then_increment()
tx.signature = user_signer.sign(tx)
proxy.send_transaction(tx)